In [1]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import sys,os,time,datetime,warnings,math,itertools

import torch
import pandas as pd

import tqdm

In [2]:
sys.path.append('../../7_HELPERFUNCTIONS/')
from WorldSimulators import TransferChannels

In [3]:
channel = TransferChannels.MIMOTimeInvariant(5,10)

In [24]:
R = np.zeros([5,10])

for k in range(10):
    v = np.zeros(10)
    v[k] = 1

    R[:,k] = channel.transfer_v(v).transpose().reshape(5)

In [70]:
R_pseudoInv = np.linalg.inv(np.einsum('ik,kj->ij',R,R.transpose()))

In [71]:
np.einsum('ii,',R_pseudoInv

array([[ 0.18374471, -0.05229017,  0.08746077, -0.04930833, -0.01648275],
       [-0.05229017,  0.33501908,  0.07544662,  0.13955541,  0.17684158],
       [ 0.08746077,  0.07544662,  0.39340113, -0.00395318,  0.05788517],
       [-0.04930833,  0.13955541, -0.00395318,  0.18510827,  0.06368441],
       [-0.01648275,  0.17684158,  0.05788517,  0.06368441,  0.1772771 ]])

In [ ]:
R.

In [410]:
class Precoder(torch.nn.Module):
    def __init__(self,channel):
        super(Precoder,self).__init__()
        self.l0 = torch.nn.Linear(channel.out_size,16)
        self.l1 = torch.nn.Linear(16,channel.in_size)
    def forward(self,x):
        x = torch.nn.functional.relu(self.l0(x))
        x = torch.nn.functional.tanh(self.l1(x))
        return x

In [429]:
precoder = Precoder(channel) 

#optimizer = torch.optim.Adam(precoder.parameters(),lr=0.01)
#criterion = torch.nn.MSELoss()

num_epochs = 5
val_size = 0.2

batch_size = 1

epsilon = 0.1

In [430]:
R = []

y_target = torch.tensor(rnd.randn(n)).float()

for k in range(1000):

    if (k+1)%100==0: print('Run {}'.format(k+1))
    
    
    x_hat = precoder(y_target)
    
    if torch.isnan(x_hat).any(): break
    
    y_measured = channel.transfer_v(np.array(x_hat.tolist()))

    penalty = np.linalg.norm(np.array(y_target)-y_measured,axis=1)
    
    c_grad = torch.autograd.grad(x_hat.sum().log(),precoder.parameters())
    for layer,p in enumerate(precoder.parameters()):
        p.data += c_grad[layer]*epsilon*penalty[0]

    R.append(penalty)

Run 100
Run 200
Run 300
Run 400
Run 500
Run 600
Run 700
Run 800
Run 900
Run 1000


In [431]:
R

[array([4.09842089]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.61922726]),
 array([5.

In [432]:
y_target

tensor([-2.1669,  0.7192, -1.1355,  1.0028, -0.5816])

In [433]:
y_measured

array([[1.89430367, 1.1793932 , 1.52419856, 0.99367888, 2.21066926]])

In [422]:
np.linalg.norm((np.array(y_target)-y_measured)[0])

6.459812584828123

In [424]:
penalty[0]

6.459812584828123